# 1. Import and Install Dependencies

In [1]:
!pip install tensorflow==2.13.0 opencv-python mediapipe scikit-learn matplotlib pygame

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


# 2. Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Collection

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['Guru1', 'Guru2', 'Guru3', 'Guru4', 'Guru5', 'Guru6', 'Guru7', 'Guru8', 'Stand'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [9]:
for action in actions: 
    #dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    #for sequence in range(1,no_sequences+1):
     for sequence in range(no_sequences):
        try: 
            #os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [10]:
pygame.init()
pygame.mixer.init()
ready_sound=pygame.mixer.Sound("getready.wav")
start_sound=pygame.mixer.Sound("startpose.wav")
end_sound=pygame.mixer.Sound("endpose.wav")

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.9) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            #ready_sound.play()
            #cv2.waitKey(3000)
            #start_sound.play()
            #cv2.waitKey(100)
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)   
                    cv2.putText(image, 'STARTING', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            #end_sound.play()
            #cv2.waitKey(3000)
        cv2.waitKey(30000)
        #temp = input("Go to the next pose")
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1711303933.488045       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [11]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [13]:
label_map = {label:num for num, label in enumerate(actions)}

In [14]:
label_map

{'Guru1': 0,
 'Guru2': 1,
 'Guru3': 2,
 'Guru4': 3,
 'Guru5': 4,
 'Guru6': 5,
 'Guru7': 6,
 'Guru8': 7,
 'Stand': 8}

In [15]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [16]:
# Ok to skip
np.array(sequences).shape

(270, 30, 1662)

In [17]:
# Ok to skip
np.array(labels).shape

(270,)

In [18]:
# Ok to skip
X = np.array(sequences)

In [19]:
# Ok to skip
X.shape

(270, 30, 1662)

In [20]:
# Ok to skip
y = to_categorical(labels).astype(int)

In [21]:
# Ok to skip
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [22]:
# Ok to skip
y_test.shape

(14, 9)

# 7. Build and Train LSTM Neural Network

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [24]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [25]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [26]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [27]:
model.fit(X_train, y_train, epochs=1500, callbacks=[tb_callback])

Epoch 1/1500
8/8 [==============================] - 2s 49ms/step - loss: 2.2410 - categorical_accuracy: 0.1289
Epoch 2/1500
8/8 [==============================] - 0s 58ms/step - loss: 2.2007 - categorical_accuracy: 0.1211
Epoch 3/1500
8/8 [==============================] - 0s 54ms/step - loss: 2.1858 - categorical_accuracy: 0.2266
Epoch 4/1500
8/8 [==============================] - 0s 59ms/step - loss: 2.1180 - categorical_accuracy: 0.2266
Epoch 5/1500
8/8 [==============================] - 0s 56ms/step - loss: 2.1452 - categorical_accuracy: 0.1680
Epoch 6/1500
8/8 [==============================] - 0s 52ms/step - loss: 1.9846 - categorical_accuracy: 0.1562
Epoch 7/1500
8/8 [==============================] - 0s 55ms/step - loss: 1.9084 - categorical_accuracy: 0.2148
Epoch 8/1500
8/8 [==============================] - 0s 55ms/step - loss: 1.9770 - categorical_accuracy: 0.2422
Epoch 9/1500
8/8 [==============================] - 0s 57ms/step - loss: 1.9366 - categorical_accuracy: 0.1172
E

8/8 [==============================] - 1s 64ms/step - loss: 13.9739 - categorical_accuracy: 0.1367
Epoch 75/1500
8/8 [==============================] - 1s 64ms/step - loss: 12.1432 - categorical_accuracy: 0.1133
Epoch 76/1500
8/8 [==============================] - 1s 67ms/step - loss: 8.4465 - categorical_accuracy: 0.0703
Epoch 77/1500
8/8 [==============================] - 0s 58ms/step - loss: 6.8299 - categorical_accuracy: 0.0898
Epoch 78/1500
8/8 [==============================] - 0s 60ms/step - loss: 5.2180 - categorical_accuracy: 0.1914
Epoch 79/1500
8/8 [==============================] - 1s 67ms/step - loss: 3.9948 - categorical_accuracy: 0.1406
Epoch 80/1500
8/8 [==============================] - 0s 57ms/step - loss: 3.1714 - categorical_accuracy: 0.1680
Epoch 81/1500
8/8 [==============================] - 0s 58ms/step - loss: 2.7785 - categorical_accuracy: 0.2539
Epoch 82/1500
8/8 [==============================] - 1s 69ms/step - loss: 2.5492 - categorical_accuracy: 0.2695
Epoc

8/8 [==============================] - 0s 63ms/step - loss: 1.1616 - categorical_accuracy: 0.5977
Epoch 147/1500
8/8 [==============================] - 0s 59ms/step - loss: 1.1618 - categorical_accuracy: 0.5664
Epoch 148/1500
8/8 [==============================] - 1s 62ms/step - loss: 1.1328 - categorical_accuracy: 0.6133
Epoch 149/1500
8/8 [==============================] - 1s 84ms/step - loss: 1.1181 - categorical_accuracy: 0.6055
Epoch 150/1500
8/8 [==============================] - 0s 58ms/step - loss: 1.1232 - categorical_accuracy: 0.6133
Epoch 151/1500
8/8 [==============================] - 0s 55ms/step - loss: 1.1124 - categorical_accuracy: 0.6055
Epoch 152/1500
8/8 [==============================] - 0s 55ms/step - loss: 1.1124 - categorical_accuracy: 0.6289
Epoch 153/1500
8/8 [==============================] - 1s 74ms/step - loss: 1.1076 - categorical_accuracy: 0.5898
Epoch 154/1500
8/8 [==============================] - 0s 56ms/step - loss: 1.1044 - categorical_accuracy: 0.628

8/8 [==============================] - 0s 57ms/step - loss: 0.7022 - categorical_accuracy: 0.7383
Epoch 219/1500
8/8 [==============================] - 0s 57ms/step - loss: 0.7132 - categorical_accuracy: 0.7227
Epoch 220/1500
8/8 [==============================] - 0s 56ms/step - loss: 0.7842 - categorical_accuracy: 0.6953
Epoch 221/1500
8/8 [==============================] - 0s 58ms/step - loss: 1.0593 - categorical_accuracy: 0.5664
Epoch 222/1500
8/8 [==============================] - 0s 57ms/step - loss: 0.9715 - categorical_accuracy: 0.6250
Epoch 223/1500
8/8 [==============================] - 0s 56ms/step - loss: 0.9171 - categorical_accuracy: 0.6328
Epoch 224/1500
8/8 [==============================] - 0s 54ms/step - loss: 0.8104 - categorical_accuracy: 0.6875
Epoch 225/1500
8/8 [==============================] - 0s 54ms/step - loss: 0.8437 - categorical_accuracy: 0.6367
Epoch 226/1500
8/8 [==============================] - 0s 54ms/step - loss: 0.7556 - categorical_accuracy: 0.734

8/8 [==============================] - 0s 58ms/step - loss: 0.9230 - categorical_accuracy: 0.6445
Epoch 291/1500
8/8 [==============================] - 1s 71ms/step - loss: 0.8698 - categorical_accuracy: 0.6484
Epoch 292/1500
8/8 [==============================] - 0s 55ms/step - loss: 0.9504 - categorical_accuracy: 0.6172
Epoch 293/1500
8/8 [==============================] - 0s 54ms/step - loss: 0.9614 - categorical_accuracy: 0.5742
Epoch 294/1500
8/8 [==============================] - 0s 55ms/step - loss: 0.9404 - categorical_accuracy: 0.6289
Epoch 295/1500
8/8 [==============================] - 1s 66ms/step - loss: 0.9658 - categorical_accuracy: 0.6250
Epoch 296/1500
8/8 [==============================] - 0s 53ms/step - loss: 0.8552 - categorical_accuracy: 0.6680
Epoch 297/1500
8/8 [==============================] - 0s 62ms/step - loss: 0.8267 - categorical_accuracy: 0.6758
Epoch 298/1500
8/8 [==============================] - 0s 54ms/step - loss: 0.8320 - categorical_accuracy: 0.656

...

8/8 [==============================] - 0s 56ms/step - loss: 1.4279 - categorical_accuracy: 0.4297
Epoch 1155/1500
8/8 [==============================] - 0s 55ms/step - loss: 1.4205 - categorical_accuracy: 0.4648
Epoch 1156/1500
8/8 [==============================] - 0s 58ms/step - loss: 1.3688 - categorical_accuracy: 0.4414
Epoch 1157/1500
8/8 [==============================] - 0s 57ms/step - loss: 1.3649 - categorical_accuracy: 0.4609
Epoch 1158/1500
8/8 [==============================] - 0s 56ms/step - loss: 1.3757 - categorical_accuracy: 0.3945
Epoch 1159/1500
8/8 [==============================] - 0s 56ms/step - loss: 1.3977 - categorical_accuracy: 0.4023
Epoch 1160/1500
8/8 [==============================] - 0s 57ms/step - loss: 1.3726 - categorical_accuracy: 0.4531
Epoch 1161/1500
8/8 [==============================] - 0s 55ms/step - loss: 1.3229 - categorical_accuracy: 0.4922
Epoch 1162/1500
8/8 [==============================] - 1s 64ms/step - loss: 1.2851 - categorical_accurac

8/8 [==============================] - 0s 58ms/step - loss: 0.9283 - categorical_accuracy: 0.6562
Epoch 1227/1500
8/8 [==============================] - 0s 63ms/step - loss: 0.9349 - categorical_accuracy: 0.6133
Epoch 1228/1500
8/8 [==============================] - 0s 61ms/step - loss: 0.9211 - categorical_accuracy: 0.6328
Epoch 1229/1500
8/8 [==============================] - 0s 60ms/step - loss: 0.8841 - categorical_accuracy: 0.6406
Epoch 1230/1500
8/8 [==============================] - 1s 65ms/step - loss: 0.8781 - categorical_accuracy: 0.6836
Epoch 1231/1500
8/8 [==============================] - 1s 68ms/step - loss: 0.7992 - categorical_accuracy: 0.6992
Epoch 1232/1500
8/8 [==============================] - 1s 66ms/step - loss: 1.0047 - categorical_accuracy: 0.6133
Epoch 1233/1500
8/8 [==============================] - 1s 65ms/step - loss: 0.9962 - categorical_accuracy: 0.6016
Epoch 1234/1500
8/8 [==============================] - 1s 68ms/step - loss: 12.1192 - categorical_accura

8/8 [==============================] - 0s 60ms/step - loss: 2.2080 - categorical_accuracy: 0.1133
Epoch 1298/1500
8/8 [==============================] - 0s 57ms/step - loss: 2.2077 - categorical_accuracy: 0.1133
Epoch 1299/1500
8/8 [==============================] - 0s 57ms/step - loss: 2.2074 - categorical_accuracy: 0.1133
Epoch 1300/1500
8/8 [==============================] - 0s 56ms/step - loss: 2.2071 - categorical_accuracy: 0.1133
Epoch 1301/1500
8/8 [==============================] - 0s 55ms/step - loss: 2.2068 - categorical_accuracy: 0.1133
Epoch 1302/1500
8/8 [==============================] - 0s 56ms/step - loss: 2.2066 - categorical_accuracy: 0.1133
Epoch 1303/1500
8/8 [==============================] - 0s 55ms/step - loss: 2.2063 - categorical_accuracy: 0.1133
Epoch 1304/1500
8/8 [==============================] - 0s 57ms/step - loss: 2.2060 - categorical_accuracy: 0.1133
Epoch 1305/1500
8/8 [==============================] - 1s 63ms/step - loss: 2.2058 - categorical_accurac

8/8 [==============================] - 0s 57ms/step - loss: 2.1982 - categorical_accuracy: 0.1133
Epoch 1370/1500
8/8 [==============================] - 0s 56ms/step - loss: 2.1981 - categorical_accuracy: 0.1133
Epoch 1371/1500
8/8 [==============================] - 0s 56ms/step - loss: 2.1981 - categorical_accuracy: 0.1133
Epoch 1372/1500
8/8 [==============================] - 0s 56ms/step - loss: 2.1981 - categorical_accuracy: 0.1133
Epoch 1373/1500
8/8 [==============================] - 0s 56ms/step - loss: 2.1980 - categorical_accuracy: 0.1133
Epoch 1374/1500
8/8 [==============================] - 1s 64ms/step - loss: 2.1979 - categorical_accuracy: 0.1133
Epoch 1375/1500
8/8 [==============================] - 1s 67ms/step - loss: 2.1979 - categorical_accuracy: 0.1133
Epoch 1376/1500
8/8 [==============================] - 1s 65ms/step - loss: 2.1979 - categorical_accuracy: 0.1133
Epoch 1377/1500
8/8 [==============================] - 0s 59ms/step - loss: 2.1979 - categorical_accurac

Epoch 1441/1500
8/8 [==============================] - 1s 64ms/step - loss: 2.1969 - categorical_accuracy: 0.1172
Epoch 1442/1500
8/8 [==============================] - 1s 66ms/step - loss: 2.1969 - categorical_accuracy: 0.1172
Epoch 1443/1500
8/8 [==============================] - 1s 78ms/step - loss: 2.1969 - categorical_accuracy: 0.1172
Epoch 1444/1500
8/8 [==============================] - 1s 65ms/step - loss: 2.1969 - categorical_accuracy: 0.1172
Epoch 1445/1500
8/8 [==============================] - 1s 71ms/step - loss: 2.1969 - categorical_accuracy: 0.1172
Epoch 1446/1500
8/8 [==============================] - 1s 71ms/step - loss: 2.1969 - categorical_accuracy: 0.1172
Epoch 1447/1500
8/8 [==============================] - 1s 64ms/step - loss: 2.1968 - categorical_accuracy: 0.1172
Epoch 1448/1500
8/8 [==============================] - 0s 59ms/step - loss: 2.1968 - categorical_accuracy: 0.1172
Epoch 1449/1500
8/8 [==============================] - 1s 79ms/step - loss: 2.1968 - cat

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 9)                 297       
                                                                 
Total params: 596873 (2.28 MB)
Trainable params: 596873 

# 8. Make Predictions

In [29]:
res = model.predict(X_test)

1/1 [==============================] - 1s 724ms/step


In [30]:
actions[np.argmax(res[2])]

'Guru8'

In [31]:
actions[np.argmax(y_test[2])]

'Guru3'

# 9. Save Weights

In [32]:
model.save('action.h5')

/Users/lpamula/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [33]:
del model

In [34]:
# Once training is done, u do not have to train it again (no need to call the fit() finction). Just need to setup the model
# and compile it. then load the saved weights.
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [35]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [36]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [37]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [38]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 349ms/step


In [39]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [40]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[11.,  0.],
        [ 3.,  0.]],

       [[13.,  0.],
        [ 1.,  0.]],

       [[13.,  0.],
        [ 1.,  0.]],

       [[13.,  0.],
        [ 1.,  0.]],

       [[12.,  0.],
        [ 2.,  0.]],

       [[12.,  0.],
        [ 2.,  0.]],

       [[13.,  0.],
        [ 1.,  0.]],

       [[ 0., 14.],
        [ 0.,  0.]],

       [[11.,  0.],
        [ 3.,  0.]]])

In [41]:
accuracy_score(ytrue, yhat)

0.0

# 11. Test in Real Time

In [42]:
from scipy import stats

In [43]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [44]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

TypeError: only size-1 arrays can be converted to Python scalars

<Figure size 1800x1800 with 0 Axes>

In [45]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        #cv2.putText(image, ' '.join(sentence), (3,30), 
        #               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1711306134.775561       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M2


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
